Import libraries:

In [ ]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
import datetime as dt
import glob

List of Excel files in current folder (that this py file is located in):

In [ ]:
inventory_list = glob.glob('*.xls*')

Define function to process the document. Adding a new column called report_generated that takes data from a time stamp in the first row.
Also re-indexing data to remove top two rows, which only holds metadata.

In [ ]:
def process(file):
    data = pd.read_excel(file)
    date = dt.datetime.strptime(data.iloc[0,0].split(' ')[2], '%d/%b/%Y')
    date = dt.datetime.strftime(date, '%m/%d/%Y')
    data.columns = data.iloc[1]
    data = data.iloc[2:,:]
    data['report_generated'] = date
    data.reset_index(inplace=True, drop=True)
    return data

Calls the previous function for all files and concatenates each time:

In [ ]:
new_inv = pd.DataFrame() #create a new empty dataframe to include all inventories list in all files
for file in inventory_list: #goes through list of file while concatenating all data together, files should have same columns
    new_inv = pd.concat([new_inv, process(file)], ignore_index=True)


Create connection to Access Database on local drive:

In [ ]:
conn =(
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    r"DBQ=C:/Users/username/Documents/file.accdb"
    r"ExtendedAnsiSQL=1;"
)
curl = sa.engine.URL.create(
    "access+pyodbc",
    query={"odbc_connect": conn}
)
engine = sa.create_engine(curl)

Append new_inv dataframe to file.accdb database:

In [ ]:
new_inv.to_sql('inventory', engine, index=False, if_exists="append")